In [314]:
import pandas as pd
import numpy as np
import datetime
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
from matplotlib.pyplot import figure
import scipy as sp
import calendar
import seaborn as sns
import os
from scipy.stats import pearsonr
from stargazer.stargazer import Stargazer
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from datetime import datetime, date, timedelta

In [315]:
#!pip install tabulate

## Digital Fundametals

In [316]:
factor_dict=pd.read_excel('C:/Users/19084/My Backup Files/Digital Assets/dict.xlsx')
factor_dict.head()

,Factor,Category,FactorNum
0,AdrActCnt,Activity,F1
1,BlkCnt,Blocks,F2
2,BlkSizeByte,Blocks,F3
3,BlkSizeMeanByte,Blocks,F4
4,CapMrktCurUSD,MarketCap,F5


# Pre-Process Data Functions

In [317]:
def factor_read(curr):
    wkdir = 'C:/Users/19084/My Backup Files/Digital Assets/'
    factor_data = wkdir+'CoinMetricsData/' + curr + '_curr.csv'
    dfname = 'factor_data'
    dfname = pd.read_csv(factor_data)
    return dfname

In [318]:
def factor_df(df_in,curr):
    df_out=pd.DataFrame(columns=['date'])
    df_out['date']=df_in['time']   
    for col in df_in.columns:
        for j in range(0,len(factor_dict)):
            if col==factor_dict['Factor'].iloc[j]:
                new_col=curr + '_' + factor_dict['FactorNum'].iloc[j]
                df_out[new_col]=df_in[col]
    return df_out           

## Read and process data

In [319]:
btc_factor_raw=factor_read('BTC')
#btc_factor_raw.head()

In [320]:
btc_factor_df=factor_df(btc_factor_raw,'BTC')
#btc_factor_df.head()

In [321]:
eth_factor_raw=factor_read('ETH')
#eth_factor_raw.head()

In [326]:
eth_factor_df=factor_df(eth_factor_raw,'ETH')
#eth_factor_df.head()

## Moving average

In [327]:
def factor_ma(df_in,ma):
    df_out=pd.DataFrame(columns=['date'])
    df_out['date']=df_in['date']
    for col in df_in.columns:
        col_name_new=col + '_MA' + str(ma)
        if col!='date':
            df_out[col_name_new]=df_in[col].rolling(ma).mean()
    return df_out

In [328]:
btc_factor_ma22=factor_ma(btc_factor_df,22)
eth_factor_ma22=factor_ma(eth_factor_df,22)

## Merge BTC-ETH

In [329]:
btc_eth_factor_df = pd.merge(btc_factor_df,eth_factor_df, on='date', how='outer')
btc_eth_factor_ma22 = pd.merge(btc_factor_ma22,eth_factor_ma22, on='date', how='outer')
btc_eth_factor_clean_full_df=btc_eth_factor_df.dropna().copy()
btc_eth_factor_clean_full_ma22=btc_eth_factor_ma22.dropna().copy()
btc_eth_factor_clean_full_df.head()

,date,BTC_F1,BTC_F2,BTC_F4,BTC_F5,BTC_F6,BTC_F7,BTC_F8,BTC_F9,BTC_F10,...,ETH_F20,ETH_F21,ETH_F22,ETH_F23,ETH_F24,ETH_F25,ETH_F26,ETH_F27,ETH_F28,ETH_F31
2438,2015-09-07,318515.0,152.0,479678.532895,3.510234e+09,5.695765e+10,0.000236,0.056684,0.0001,0.024063,...,7.306705e+07,5702.0,5506.0,1.439252e+05,1.835924e+05,36.823785,46.972788,2.087818,2.663241,0.127936
2439,2015-09-08,338676.0,136.0,703476.838235,3.562724e+09,5.695765e+10,0.000205,0.050073,0.0001,0.024417,...,7.309293e+07,5622.0,5570.0,2.232403e+05,2.820627e+05,47.344776,59.819830,2.393153,3.023734,0.127953
2440,2015-09-09,298432.0,149.0,730847.134228,3.484056e+09,5.695765e+10,0.000217,0.051766,0.0001,0.023872,...,7.311937e+07,5604.0,5531.0,1.112715e+06,1.346385e+06,207.924885,251.589110,2.018485,2.442367,0.128217
2441,2015-09-10,283028.0,171.0,682503.847953,3.485486e+09,5.695765e+10,0.000221,0.052781,0.0001,0.023875,...,7.314581e+07,5341.0,5243.0,1.385908e+05,1.643271e+05,39.296958,46.594403,2.404459,2.850967,0.123046
2442,2015-09-11,317585.0,158.0,713392.594937,3.517014e+09,5.695765e+10,0.000212,0.051100,0.0001,0.024085,...,7.317305e+07,6460.0,6319.0,5.759335e+05,5.343026e+05,115.064924,106.747538,2.078264,1.928038,0.121418


# Prices

In [330]:
def price_returns(curr):
    wkdir = 'C:/Users/19084/My Backup Files/Digital Assets/'
    price_data = wkdir+'CoinMetricsData/' + curr + '_curr.csv'
    temp = pd.read_csv(price_data)
    curr_price = curr+'_USD'
    df_out=pd.DataFrame(columns=['date',curr_price])
    df_out['date']=temp['time']
    df_out[curr_price]=temp['PriceUSD']
    df_out.dropna(inplace=True)
    curr_ret = curr + '_Ret'
    df_out[curr_ret]=df_out[curr_price].pct_change()
    df_clean=df_out[['date',curr_price,curr_ret]].copy()
    return df_clean
    # return df_out

In [331]:
def price_returns_m(curr):
    wkdir = 'C:/Users/19084/My Backup Files/Digital Assets/'
    price_data = wkdir+'CoinMetricsData/' + curr + '_curr.csv'
    temp = pd.read_csv(price_data)
    curr_price = curr+'_USD'
    df_out=pd.DataFrame(columns=['date',curr_price])
    df_out['date']=temp['time']
    df_out[curr_price]=temp['PriceUSD']
    df_out.dropna(inplace=True)
    df_out['month']= pd.DatetimeIndex(df_out['date']).month
    df_out['lmonth']=df_out['month'].shift(-1)
    df_out['diffmonth']=df_out['month']-df_out['lmonth']
    df_out['dmonth']=df_out['diffmonth'].apply(lambda x: 1 if x!=0 else 0)
    curr_eom = curr_price + '_EoM'
    df_out[curr_eom]=df_out['dmonth']*df_out[curr_price]
    eom_df=df_out[df_out[curr_eom]>0]
    curr_ret = curr + '_Ret'
    eom_df[curr_ret]=eom_df[curr_eom].pct_change()
    eom_df_clean=eom_df[['date',curr_price,curr_ret]].copy()
    return eom_df_clean
    # return df_out

In [332]:
btc_price=price_returns('BTC')
eth_price=price_returns('ETH')
btc_price_m=price_returns_m('BTC')
eth_price_m=price_returns_m('ETH')

C:\Users\19084\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [333]:
eth_price.head()

,date,ETH_USD,ETH_Ret
9,2015-08-08,1.19999,NaN
10,2015-08-09,1.19999,0.000000
11,2015-08-10,1.19999,0.000000
12,2015-08-11,0.99000,-0.174993
13,2015-08-12,1.28800,0.301010


## Merge BTC-ETH prices and returns

In [334]:
btc_eth_price=pd.merge(btc_price,eth_price, on='date', how='outer')
btc_eth_price_clean=btc_eth_price.dropna()
btc_eth_price_clean['BTC_ETH_Ret']=btc_eth_price_clean['BTC_Ret']-btc_eth_price_clean['ETH_Ret']
btc_eth_price_clean.head()

C:\Users\19084\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret
1848,2015-08-09,266.342020,0.018710,1.19999,0.000000,0.018710
1849,2015-08-10,264.928825,-0.005306,1.19999,0.000000,-0.005306
1850,2015-08-11,271.421736,0.024508,0.99000,-0.174993,0.199501
1851,2015-08-12,268.143868,-0.012077,1.28800,0.301010,-0.313087
1852,2015-08-13,264.691628,-0.012875,1.88480,0.463354,-0.476229


In [335]:
btc_eth_price_m=pd.merge(btc_price_m,eth_price_m, on='date', how='outer')
btc_eth_price_m_clean=btc_eth_price_m.dropna()
btc_eth_price_m_clean['BTC_ETH_Ret']=btc_eth_price_m_clean['BTC_Ret']-btc_eth_price_m_clean['ETH_Ret']
btc_eth_price_m_clean.head()

C:\Users\19084\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret
62,2015-09-30,236.739848,0.026227,0.713644,-0.453778,0.480006
63,2015-10-31,314.119508,0.326855,0.889343,0.246199,0.080657
64,2015-11-30,377.389840,0.201421,0.880000,-0.010505,0.211926
65,2015-12-31,429.677153,0.138550,0.948840,0.078227,0.060323
66,2016-01-31,366.341688,-0.147402,2.242672,1.363593,-1.510996


In [336]:
btc_eth_model_m=pd.merge(btc_eth_price_m_clean,btc_eth_factor_clean_full_ma22, on='date', how='outer')
btc_eth_model_m_keep=btc_eth_model_m.dropna()
btc_eth_model_m=btc_eth_model_m_keep
btc_eth_model_m.head()

,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,BTC_F1_MA22,BTC_F2_MA22,BTC_F4_MA22,BTC_F5_MA22,...,ETH_F20_MA22,ETH_F21_MA22,ETH_F22_MA22,ETH_F23_MA22,ETH_F24_MA22,ETH_F25_MA22,ETH_F26_MA22,ETH_F27_MA22,ETH_F28_MA22,ETH_F31_MA22
0,2015-09-30,236.739848,0.026227,0.713644,-0.453778,0.480006,313073.136364,148.818182,570940.558828,3.422213e+09,...,7.339591e+07,5939.727273,5809.818182,360114.289817,3.234671e+05,76.520689,67.992275,1.973719,1.741110,0.107434
1,2015-10-31,314.119508,0.326855,0.889343,0.246199,0.080657,350885.363636,149.136364,557225.532919,4.068208e+09,...,7.420567e+07,6649.000000,6327.772727,471823.955653,3.448106e+05,91.897425,65.094788,2.122669,1.430867,0.098419
2,2015-11-30,377.389840,0.201421,0.880000,-0.010505,0.211926,383127.136364,156.863636,543914.617522,5.031982e+09,...,7.502103e+07,7637.954545,6856.545455,494697.325521,4.589952e+05,78.371142,72.694852,2.271409,2.068733,0.115579
3,2015-12-31,429.677153,0.138550,0.948840,0.078227,0.060323,456927.772727,164.363636,629535.424521,6.608428e+09,...,7.585847e+07,12872.772727,10128.181818,289677.091930,2.629607e+05,34.589904,31.369863,1.189122,1.065410,0.047791
4,2016-01-31,366.341688,-0.147402,2.242672,1.363593,-1.510996,488123.454545,158.727273,693919.179089,6.031319e+09,...,7.669212e+07,13199.818182,11669.045455,808837.792942,1.533407e+06,87.590428,158.919503,1.376002,2.316887,0.056826


In [337]:
btc_eth_model=pd.merge(btc_eth_price_clean,btc_eth_factor_clean_full_df, on='date', how='outer')
btc_eth_model_keep=btc_eth_model.dropna()
btc_eth_model=btc_eth_model_keep
btc_eth_model.head()

,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,BTC_F1,BTC_F2,BTC_F4,BTC_F5,...,ETH_F20,ETH_F21,ETH_F22,ETH_F23,ETH_F24,ETH_F25,ETH_F26,ETH_F27,ETH_F28,ETH_F31
29,2015-09-07,240.633616,-0.000714,1.275610,-0.048052,0.047339,318515.0,152.0,479678.532895,3.510234e+09,...,7.306705e+07,5702.0,5506.0,1.439252e+05,1.835924e+05,36.823785,46.972788,2.087818,2.663241,0.127936
30,2015-09-08,244.174953,0.014717,1.263494,-0.009498,0.024215,338676.0,136.0,703476.838235,3.562724e+09,...,7.309293e+07,5622.0,5570.0,2.232403e+05,2.820627e+05,47.344776,59.819830,2.393153,3.023734,0.127953
31,2015-09-09,238.722463,-0.022330,1.210000,-0.042338,0.020008,298432.0,149.0,730847.134228,3.484056e+09,...,7.311937e+07,5604.0,5531.0,1.112715e+06,1.346385e+06,207.924885,251.589110,2.018485,2.442367,0.128217
32,2015-09-10,238.750477,0.000117,1.185700,-0.020083,0.020200,283028.0,171.0,682503.847953,3.485486e+09,...,7.314581e+07,5341.0,5243.0,1.385908e+05,1.643271e+05,39.296958,46.594403,2.404459,2.850967,0.123046
33,2015-09-11,240.845008,0.008773,0.927716,-0.217580,0.226353,317585.0,158.0,713392.594937,3.517014e+09,...,7.317305e+07,6460.0,6319.0,5.759335e+05,5.343026e+05,115.064924,106.747538,2.078264,1.928038,0.121418


In [339]:
btc_eth_model.to_csv('btc_eth_model.csv',index=False)
btc_eth_model.to_excel('btc_eth_model.xlsx',index=False)
btc_eth_model_m.to_csv('btc_eth_model_m.csv',index=False)
btc_eth_model_m.to_excel('btc_eth_model_m.xlsx',index=False)

## Lag the factors

In [340]:
def factor_lag(df_in,first_col,last_col):
    df_out=df_in.iloc[:,0:first_col]
    j=first_col
    for j in range(first_col,last_col):
        col_name_old=df_in.columns[j]
        col_name_new='L_'+col_name_old
        df_out[col_name_new]=df_in.iloc[:,j].shift(1)
        j=j+1
    return df_out

In [343]:
def relative_factors_m(df_in,first_col,curr1,curr2):
    df_out=df_in.iloc[:,0:first_col]
    for j in range(1,32):
        curr1_col=curr1 + '_F' + str(j) + '_MA22'
        curr2_col=curr2 + '_F' + str(j) + '_MA22'    
        new_col=curr1 + '_' + curr2 + '_F' + str(j) + '_ratio'
        try:
            df_out[new_col]=df_in[curr1_col]/df_in[curr2_col]
        except:
            continue
    return df_out

In [355]:
def relative_factors_m_l(df_in,first_col,curr1,curr2):
    df_out=df_in.iloc[:,0:first_col]
    for j in range(1,32):
        curr1_col='L_' + curr1 + '_F' + str(j) + '_MA22'
        curr2_col='L_' + curr2 + '_F' + str(j) + '_MA22'    
        new_col='L_' + curr1 + '_' + curr2 + '_F' + str(j) + '_ratio'
        try:
            df_out[new_col]=df_in[curr1_col]/df_in[curr2_col]
        except:
            continue
    return df_out

In [356]:
def relative_factors_l(df_in,first_col,curr1,curr2):
    df_out=df_in.iloc[:,0:first_col]
    for j in range(1,32):
        curr1_col='L_' + curr1 + '_F' + str(j)
        curr2_col='L_' + curr2 + '_F' + str(j)   
        new_col='L_' + curr1 + '_' + curr2 + '_F' + str(j) + '_ratio'
        try:
            df_out[new_col]=df_in[curr1_col]/df_in[curr2_col]
        except:
            continue
    return df_out

In [346]:
def relative_factors(df_in,first_col,curr1,curr2):
    df_out=df_in.iloc[:,0:first_col]
    for j in range(1,32):
        curr1_col=curr1 + '_F' + str(j)
        curr2_col=curr2 + '_F' + str(j)   
        new_col=curr1 + '_' + curr2 + '_F' + str(j) + '_ratio'
        try:
            df_out[new_col]=df_in[curr1_col]/df_in[curr2_col]
        except:
            continue
    return df_out

In [347]:
btc_eth_model=pd.read_excel('btc_eth_model.xlsx')
btc_eth_model_m=pd.read_excel('btc_eth_model_m.xlsx')

In [348]:
btc_eth_model_lags=factor_lag(btc_eth_model,6,61)
btc_eth_model_lags_m=factor_lag(btc_eth_model_m,6,61)

btc_eth_model_lags_keep=btc_eth_model_lags.dropna()
btc_eth_model_lags_keep_m=btc_eth_model_lags_m.dropna()

btc_eth_model_lags=btc_eth_model_lags_keep
btc_eth_model_lags_m=btc_eth_model_lags_keep_m

btc_eth_price_clean=btc_eth_model_lags[['date','BTC_USD','BTC_Ret','ETH_USD','ETH_Ret','BTC_ETH_Ret']].copy()
btc_eth_price_clean_m=btc_eth_model_lags_m[['date','BTC_USD','BTC_Ret','ETH_USD','ETH_Ret','BTC_ETH_Ret']].copy()

btc_eth_model_lags.to_csv('btc_eth_model_lags.csv',index=False)
btc_eth_model_lags_m.to_csv('btc_eth_model_lags_m.csv',index=False)

In [360]:
#btc_eth_model_lags.head()

In [357]:
btc_eth_factor_ratios=relative_factors(btc_eth_model,6,'BTC','ETH')
btc_eth_factor_ratios_lags=relative_factors_l(btc_eth_model_lags,6,'BTC','ETH')
btc_eth_factor_ratios_m=relative_factors_m(btc_eth_model_m,6,'BTC','ETH')
btc_eth_factor_ratios_lags_m=relative_factors_m_l(btc_eth_model_lags_m,6,'BTC','ETH')

In [361]:
#btc_eth_factor_ratios_lags_m.head()

## Subset the data

In [295]:
def subset_dates(df_in):
    start=datetime.strptime('2020-12-31',"%Y-%m-%d")
    end=datetime.strptime('2022-06-30',"%Y-%m-%d")
    df_in['date'] = df_in['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
    df_out=df_in.loc[(df_in['date'] > start) & (df_in['date']  <= end)]
    df_out['date'] = df_out['date'].apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))
    return df_out


#start=datetime.strptime('2020-12-31',"%Y-%m-%d")
#end=datetime.strptime('2022-06-30',"%Y-%m-%d")
#btc_eth_factor_clean_full_df['date'] = btc_eth_factor_clean_full_df['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
#btc_eth_factor_clean_df=btc_eth_factor_clean_full_df.loc[(btc_eth_factor_clean_full_df['date'] > start) & (btc_eth_factor_clean_full_df['date']  <= end)]
#btc_eth_factor_clean_df['date'] = btc_eth_factor_clean_df['date'].apply(lambda x: datetime.strftime(x, "%Y-%m-%d"))

In [362]:
btc_eth_model = subset_dates(btc_eth_model)
btc_eth_model_lags = subset_dates(btc_eth_model_lags)
btc_eth_model_m = subset_dates(btc_eth_model_m)
btc_eth_model_lags_m = subset_dates(btc_eth_model_lags_m)
btc_eth_factor_ratios = subset_dates(btc_eth_factor_ratios)
btc_eth_factor_ratios_lags = subset_dates(btc_eth_factor_ratios_lags)
btc_eth_factor_ratios_m = subset_dates(btc_eth_factor_ratios_m)
btc_eth_factor_ratios_lags_m = subset_dates(btc_eth_factor_ratios_lags_m)

C:\Users\19084\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [389]:
btc_eth_factor_ratios_lags_m.head()

,date,BTC_USD,BTC_Ret,ETH_USD,ETH_Ret,BTC_ETH_Ret,L_BTC_ETH_F1_ratio,L_BTC_ETH_F2_ratio,L_BTC_ETH_F4_ratio,L_BTC_ETH_F5_ratio,...,L_BTC_ETH_F19_ratio,L_BTC_ETH_F20_ratio,L_BTC_ETH_F21_ratio,L_BTC_ETH_F22_ratio,L_BTC_ETH_F23_ratio,L_BTC_ETH_F24_ratio,L_BTC_ETH_F25_ratio,L_BTC_ETH_F26_ratio,L_BTC_ETH_F27_ratio,L_BTC_ETH_F28_ratio
64,2021-01-31,33157.832575,0.142480,1317.346364,0.782544,-0.640064,2.068242,0.022469,30.096577,5.944207,...,2.456407,0.163066,0.264346,1.302546,0.066970,2.442373,0.072026,2.618556,0.041512,1.489666
65,2021-02-28,45359.464164,0.367986,1425.385867,0.082013,0.285973,1.982927,0.022442,29.923052,4.460394,...,3.959324,0.162724,0.264067,1.286426,0.042349,1.168089,0.049493,1.369392,0.029048,0.796929
66,2021-03-31,58792.194828,0.296140,1916.880019,0.344815,-0.048675,1.895890,0.022157,29.192240,4.590068,...,3.684748,0.162410,0.248829,1.206553,0.044601,1.277804,0.059338,1.703799,0.020951,0.589821
67,2021-04-30,57741.444417,-0.017872,2770.290058,0.445208,-0.463080,1.943857,0.023103,28.846836,5.193959,...,2.875711,0.162063,0.234563,1.121717,0.056496,1.804637,0.086109,2.749334,0.020223,0.646945
68,2021-05-31,37312.974897,-0.353792,2699.214458,-0.025656,-0.328136,1.491869,0.020469,26.784431,3.856577,...,2.568413,0.161727,0.191764,0.938528,0.062404,1.466633,0.127870,2.973595,0.033666,0.800045


# Daily Models

### Non-Lagged Non-Ratio Models

In [364]:
lm_string_BTC='BTC_Ret ~ BTC_F1 + BTC_F2 + BTC_F4 + BTC_F6 + BTC_F13 + BTC_F14 + BTC_F20 + BTC_F21'
lm_string_ETH='ETH_Ret ~ ETH_F1 + ETH_F2 + ETH_F4 + ETH_F6 + ETH_F13 + ETH_F14 + ETH_F20 + ETH_F21 + ETH_F22'
print(lm_string_BTC)
print(lm_string_ETH)

BTC_Ret ~ BTC_F1 + BTC_F2 + BTC_F4 + BTC_F6 + BTC_F13 + BTC_F14 + BTC_F20 + BTC_F21
ETH_Ret ~ ETH_F1 + ETH_F2 + ETH_F4 + ETH_F6 + ETH_F13 + ETH_F14 + ETH_F20 + ETH_F21 + ETH_F22


In [365]:
model_BTC=smf.ols(lm_string_BTC, data = btc_eth_model).fit().summary()
print(model_BTC)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     2.325
Date:                Fri, 30 Sep 2022   Prob (F-statistic):             0.0317
Time:                        14:27:56   Log-Likelihood:                 990.74
No. Observations:                 546   AIC:                            -1967.
Df Residuals:                     539   BIC:                            -1937.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.455e-10   2.01e-09     -0.172      0.8

In [366]:
model_ETH=smf.ols(lm_string_ETH, data = btc_eth_model).fit().summary()
print(model_ETH)

                            OLS Regression Results                            
Dep. Variable:                ETH_Ret   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     4.052
Date:                Fri, 30 Sep 2022   Prob (F-statistic):           0.000552
Time:                        14:27:56   Log-Likelihood:                 848.95
No. Observations:                 546   AIC:                            -1684.
Df Residuals:                     539   BIC:                            -1654.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.343e-09   4.37e-09     -0.307      0.7

### Lagged Non-Ratio Models

In [367]:
lm_string_BTC='BTC_Ret ~ L_BTC_F1 + L_BTC_F2 + L_BTC_F4 + L_BTC_F6 + L_BTC_F13 + L_BTC_F14 + L_BTC_F20 + L_BTC_F21'
lm_string_ETH='ETH_Ret ~ L_ETH_F1 + L_ETH_F2 + L_ETH_F4 + L_ETH_F6 + L_ETH_F13 + L_ETH_F14 + L_ETH_F20 + L_ETH_F21 + L_ETH_F22'
print(lm_string_BTC)
print(lm_string_ETH)

BTC_Ret ~ L_BTC_F1 + L_BTC_F2 + L_BTC_F4 + L_BTC_F6 + L_BTC_F13 + L_BTC_F14 + L_BTC_F20 + L_BTC_F21
ETH_Ret ~ L_ETH_F1 + L_ETH_F2 + L_ETH_F4 + L_ETH_F6 + L_ETH_F13 + L_ETH_F14 + L_ETH_F20 + L_ETH_F21 + L_ETH_F22


In [368]:
lag_model_BTC=smf.ols(lm_string_BTC, data = btc_eth_model_lags).fit().summary()
print(lag_model_BTC)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.430
Date:                Fri, 30 Sep 2022   Prob (F-statistic):              0.201
Time:                        14:28:02   Log-Likelihood:                 988.08
No. Observations:                 546   AIC:                            -1962.
Df Residuals:                     539   BIC:                            -1932.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.584e-09   1.98e-09      0.801      0.4

In [369]:
lag_model_ETH=smf.ols(lm_string_ETH, data = btc_eth_model_lags).fit().summary()
print(lag_model_ETH)

                            OLS Regression Results                            
Dep. Variable:                ETH_Ret   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     2.913
Date:                Fri, 30 Sep 2022   Prob (F-statistic):            0.00837
Time:                        14:28:07   Log-Likelihood:                 845.61
No. Observations:                 546   AIC:                            -1677.
Df Residuals:                     539   BIC:                            -1647.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -4.288e-09   4.41e-09     -0.973      0.3

## Daily Ratio Models

### Ratio Non-Lagged Model

In [371]:
lm_string_BTC_ETH_Ratio='BTC_Ret ~ BTC_ETH_F1_ratio + BTC_ETH_F2_ratio + BTC_ETH_F4_ratio + BTC_ETH_F6_ratio + BTC_ETH_F13_ratio + BTC_ETH_F14_ratio + BTC_ETH_F20_ratio + BTC_ETH_F21_ratio'
print(lm_string_BTC_ETH_Ratio)

BTC_Ret ~ BTC_ETH_F1_ratio + BTC_ETH_F2_ratio + BTC_ETH_F4_ratio + BTC_ETH_F6_ratio + BTC_ETH_F13_ratio + BTC_ETH_F14_ratio + BTC_ETH_F20_ratio + BTC_ETH_F21_ratio


In [372]:
ratio_model=smf.ols(lm_string_BTC_ETH_Ratio, data = btc_eth_factor_ratios).fit().summary()
print(ratio_model)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     2.050
Date:                Fri, 30 Sep 2022   Prob (F-statistic):             0.0391
Time:                        14:28:23   Log-Likelihood:                 991.98
No. Observations:                 546   AIC:                            -1966.
Df Residuals:                     537   BIC:                            -1927.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             1.3702      1.84

### Ratio Lagged Model

In [373]:
lm_string_BTC_ETH_Ratio='BTC_Ret ~ L_BTC_ETH_F1_ratio + L_BTC_ETH_F2_ratio + L_BTC_ETH_F4_ratio + L_BTC_ETH_F6_ratio + L_BTC_ETH_F13_ratio + L_BTC_ETH_F14_ratio + L_BTC_ETH_F20_ratio + L_BTC_ETH_F21_ratio'
print(lm_string_BTC_ETH_Ratio)

BTC_Ret ~ L_BTC_ETH_F1_ratio + L_BTC_ETH_F2_ratio + L_BTC_ETH_F4_ratio + L_BTC_ETH_F6_ratio + L_BTC_ETH_F13_ratio + L_BTC_ETH_F14_ratio + L_BTC_ETH_F20_ratio + L_BTC_ETH_F21_ratio


In [374]:
ratio_lag_model=smf.ols(lm_string_BTC_ETH_Ratio, data = btc_eth_factor_ratios_lags).fit().summary()
print(ratio_lag_model)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.6984
Date:                Fri, 30 Sep 2022   Prob (F-statistic):              0.693
Time:                        14:28:33   Log-Likelihood:                 986.59
No. Observations:                 546   AIC:                            -1955.
Df Residuals:                     537   BIC:                            -1916.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.9474    

# Monthly Models

### Non-Lagged Non-Ratio Models

In [383]:
lm_string_BTC_m='BTC_Ret ~ BTC_F1_MA22 + BTC_F2_MA22 + BTC_F4_MA22 + BTC_F6_MA22 + BTC_F13_MA22 + BTC_F14_MA22 + BTC_F20_MA22 + BTC_F21_MA22'
lm_string_ETH_m='ETH_Ret ~ ETH_F1_MA22 + ETH_F2_MA22 + ETH_F4_MA22 + ETH_F6_MA22 + ETH_F13_MA22 + ETH_F14_MA22 + ETH_F20_MA22 + ETH_F21_MA22 + ETH_F22_MA22'
print(lm_string_BTC_m)
print(lm_string_ETH_m)

BTC_Ret ~ BTC_F1_MA22 + BTC_F2_MA22 + BTC_F4_MA22 + BTC_F6_MA22 + BTC_F13_MA22 + BTC_F14_MA22 + BTC_F20_MA22 + BTC_F21_MA22
ETH_Ret ~ ETH_F1_MA22 + ETH_F2_MA22 + ETH_F4_MA22 + ETH_F6_MA22 + ETH_F13_MA22 + ETH_F14_MA22 + ETH_F20_MA22 + ETH_F21_MA22 + ETH_F22_MA22


In [384]:
model_BTC_m=smf.ols(lm_string_BTC_m, data = btc_eth_model_m).fit().summary()
print(model_BTC_m)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.647
Model:                            OLS   Adj. R-squared:                  0.454
Method:                 Least Squares   F-statistic:                     3.356
Date:                Fri, 30 Sep 2022   Prob (F-statistic):             0.0393
Time:                        14:33:42   Log-Likelihood:                 10.891
No. Observations:                  18   AIC:                            -7.783
Df Residuals:                      11   BIC:                            -1.550
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     1.426e-07      1e-07      1.421   

C:\Users\19084\Anaconda3\lib\site-packages\scipy\stats\stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  "anyway, n=%i" % int(n))


In [385]:
model_ETH_m=smf.ols(lm_string_ETH, data = btc_eth_model_m).fit().summary()
print(model_ETH_m)

                            OLS Regression Results                            
Dep. Variable:                ETH_Ret   R-squared:                       0.563
Model:                            OLS   Adj. R-squared:                  0.324
Method:                 Least Squares   F-statistic:                     2.358
Date:                Fri, 30 Sep 2022   Prob (F-statistic):              0.103
Time:                        14:33:46   Log-Likelihood:                 3.1413
No. Observations:                  18   AIC:                             7.717
Df Residuals:                      11   BIC:                             13.95
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     1.461e-09   1.36e-09      1.074   

C:\Users\19084\Anaconda3\lib\site-packages\scipy\stats\stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  "anyway, n=%i" % int(n))


### Non-Lagged Ratio Models

In [386]:
lm_string_BTC_m='BTC_Ret ~ L_BTC_F1_MA22 + L_BTC_F2_MA22 + L_BTC_F4_MA22 + L_BTC_F6_MA22 + L_BTC_F13_MA22 + L_BTC_F14_MA22 + L_BTC_F20_MA22 + L_BTC_F21_MA22'
lm_string_ETH_m='ETH_Ret ~ L_ETH_F1_MA22 + L_ETH_F2_MA22 + L_ETH_F4_MA22 + L_ETH_F6_MA22 + L_ETH_F13_MA22 + L_ETH_F14_MA22 + L_ETH_F20_MA22 + L_ETH_F21_MA22 + L_ETH_F22_MA22'
print(lm_string_BTC_m)
print(lm_string_ETH_m)

BTC_Ret ~ L_BTC_F1_MA22 + L_BTC_F2_MA22 + L_BTC_F4_MA22 + L_BTC_F6_MA22 + L_BTC_F13_MA22 + L_BTC_F14_MA22 + L_BTC_F20_MA22 + L_BTC_F21_MA22
ETH_Ret ~ L_ETH_F1_MA22 + L_ETH_F2_MA22 + L_ETH_F4_MA22 + L_ETH_F6_MA22 + L_ETH_F13_MA22 + L_ETH_F14_MA22 + L_ETH_F20_MA22 + L_ETH_F21_MA22 + L_ETH_F22_MA22


In [387]:
lag_model_BTC_m=smf.ols(lm_string_BTC_m, data = btc_eth_model_lags_m).fit().summary()
print(lag_model_BTC_m)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.476
Model:                            OLS   Adj. R-squared:                  0.190
Method:                 Least Squares   F-statistic:                     1.663
Date:                Fri, 30 Sep 2022   Prob (F-statistic):              0.220
Time:                        14:35:35   Log-Likelihood:                 7.3366
No. Observations:                  18   AIC:                           -0.6733
Df Residuals:                      11   BIC:                             5.559
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept      -1.221e-07   9.07e-08     -1.

C:\Users\19084\Anaconda3\lib\site-packages\scipy\stats\stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  "anyway, n=%i" % int(n))


In [388]:
lag_model_ETH_m=smf.ols(lm_string_ETH_m, data = btc_eth_model_lags_m).fit().summary()
print(lag_model_ETH_m)

                            OLS Regression Results                            
Dep. Variable:                ETH_Ret   R-squared:                       0.635
Model:                            OLS   Adj. R-squared:                  0.437
Method:                 Least Squares   F-statistic:                     3.196
Date:                Fri, 30 Sep 2022   Prob (F-statistic):             0.0455
Time:                        14:36:02   Log-Likelihood:                 4.7804
No. Observations:                  18   AIC:                             4.439
Df Residuals:                      11   BIC:                             10.67
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       4.559e-09   4.37e-09      1.

C:\Users\19084\Anaconda3\lib\site-packages\scipy\stats\stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  "anyway, n=%i" % int(n))


## Monthly Ratio Models

### Ratio Non-Lagged Model

In [392]:
lm_string_BTC_ETH_Ratio_m='BTC_Ret ~ BTC_ETH_F1_ratio + BTC_ETH_F2_ratio + BTC_ETH_F4_ratio + BTC_ETH_F6_ratio + BTC_ETH_F13_ratio + BTC_ETH_F14_ratio + BTC_ETH_F20_ratio + BTC_ETH_F21_ratio'
print(lm_string_BTC_ETH_Ratio_m)

BTC_Ret ~ BTC_ETH_F1_ratio + BTC_ETH_F2_ratio + BTC_ETH_F4_ratio + BTC_ETH_F6_ratio + BTC_ETH_F13_ratio + BTC_ETH_F14_ratio + BTC_ETH_F20_ratio + BTC_ETH_F21_ratio


In [393]:
ratio_model_m=smf.ols(lm_string_BTC_ETH_Ratio_m, data = btc_eth_factor_ratios_m).fit().summary()
print(ratio_model_m)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.402
Model:                            OLS   Adj. R-squared:                 -0.130
Method:                 Least Squares   F-statistic:                    0.7548
Date:                Fri, 30 Sep 2022   Prob (F-statistic):              0.649
Time:                        14:39:34   Log-Likelihood:                 6.1467
No. Observations:                  18   AIC:                             5.707
Df Residuals:                       9   BIC:                             13.72
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            44.5553     92.34

C:\Users\19084\Anaconda3\lib\site-packages\scipy\stats\stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  "anyway, n=%i" % int(n))


### Ratio Lagged Model

In [394]:
lm_string_BTC_ETH_Ratio_m='BTC_Ret ~ L_BTC_ETH_F1_ratio + L_BTC_ETH_F2_ratio + L_BTC_ETH_F4_ratio + L_BTC_ETH_F6_ratio + L_BTC_ETH_F13_ratio + L_BTC_ETH_F14_ratio + L_BTC_ETH_F20_ratio + L_BTC_ETH_F21_ratio'
print(lm_string_BTC_ETH_Ratio_m)

BTC_Ret ~ L_BTC_ETH_F1_ratio + L_BTC_ETH_F2_ratio + L_BTC_ETH_F4_ratio + L_BTC_ETH_F6_ratio + L_BTC_ETH_F13_ratio + L_BTC_ETH_F14_ratio + L_BTC_ETH_F20_ratio + L_BTC_ETH_F21_ratio


In [395]:
ratio_lag_model_m=smf.ols(lm_string_BTC_ETH_Ratio_m, data = btc_eth_factor_ratios_lags_m).fit().summary()
print(ratio_lag_model_m)

                            OLS Regression Results                            
Dep. Variable:                BTC_Ret   R-squared:                       0.485
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     1.058
Date:                Fri, 30 Sep 2022   Prob (F-statistic):              0.463
Time:                        14:40:39   Log-Likelihood:                 7.4916
No. Observations:                  18   AIC:                             3.017
Df Residuals:                       9   BIC:                             11.03
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              51.7346    

C:\Users\19084\Anaconda3\lib\site-packages\scipy\stats\stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=18
  "anyway, n=%i" % int(n))
